<a href="https://colab.research.google.com/github/richtong/src/blob/rich-covid/Restart_NYC_2020_05_30%20-%202020-05-31%20branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_Proprietary and Confidential. Do not distribute without permission._

---

# Restart Partners  

---
_To:_ Jun Amora (Mayors Office, City of New York)  
_From:_ Bharat Shyam, Rich Tong (Restart)  
_Re:_ Analysis for NYC PPE needs  
_Date:_ 20 May 2020  

--- 

New York City needs a 90-day stockpile for the heathcare workers, first responders and congregate care facilities is really important, but coming up with an estimate for this is difficult given the variability of the infection and the uncertainty in the degree of economic recovery and social mobility. Therefore, we are providing another projection to augment yours that shows that our figures are within 30%-50% of your bottoms estimate. Given that we are happy to:

- _Refine healthcare estimates_. All models are heavily dependent on estimates of population involved and usage data. 
- _Non-healthcare estimates_. For instance, this model does project needs outside of the healthcare area such as small business, vertical industries and vulnerable populations.
- _Long-term modeling_. We are extending the model to include test equipment, disinfectant wipes and liquid disinfectants, so happy to add things that you need. Also we will be integrating epi and economic models too and would love to partner with you on that.

Given the uncertainties involved, this might help you make the right estimates. What follows next are:

1. Disclaimer. This is not a definitive estimate. You should use other sources and information to make your decisions.
2. Data Sources. We have included the model source data, how the model is constructed and then results. Feel free to use this data and modify as appropriate, but it serves as documentation for all the assumptions made.
3. Model. The way the calculation is done with assumptions and resulting projections
4. Outputs. The conclusions we can draw from the projection.

## Disclaimer
It must be noted that the Restart Partners ("Restart") Equipment Model (the "Model") is made available for public use free of charge. Determining equipment needs for each jurisdiction, entity or other party (each a "User") is a complex and multifaceted decision process. Restart does not does not have the authority or ability to assign empirical risks levels nor make definitive use decisions for any User. Rather, the Model provides one approach to making recommendations that can help Users make decisions about their potential equipment uses by allowing them to calculate their potential requirements. Users are strongly encouraged to consider other sources of information and expressly disclaim any cause of action they may have against Restart arising from or relating to the Model or its analysis. Implementing the equipment levels projected by the Model will not eliminate the risk of COID-19 cases being linked to activites in an economy or workplace. In this context, it is important to note that this equipment alone will not eliminate the risk of infection. All Users should remain informed about and abide by any decisions made by local public health and government authorities regarding specific mitigation efforts, including equipment in the model, as the situation is dynamic.

# Model Data

Because we do not have New York City specific data, we used various open data sources to fill in the five major assumptions in the model:

1. Usage by Population. This cuts the item usage per person per day. This right now is a series of levels. So we have four levels for civilians and then two levels for healthcare workers.
2. Usage per Patient. This is the way Epidemiological models work. That is, given a number of patients, calculate how much they will need. The model currently uses the [WHO Surge Essential Supplies Forecasting Tool v1.2](https://www.who.int/emergencies/diseases/novel-coronavirus-2019/technical-guidance/covid-19-critical-items) and estimates the entire US population use with 1,000 cases and fast transmission and slow response. So this is a very pessimistic scenario. This makes sense when calculating the surge estimates.

# Strategy for the Model

The next steps are a little complicated, but we are converting the entire model into a series of vectorized operations. 

## Daily Usage of Equipment by Protection levels D[l, n]

So first we need the inputs which are the list of protection levels p x the number of items we are tracking n. So this is an p x n matrix where each entry says for protection level l, we have so many items per capita per day. So the first data list is Daily_usage D = l rows x n columns or D.shape = ( l, n )

In Python speak this is Usage_pd since it is a _P_anda _D_ataframe. Right now this is a test matrix that is constructed in the Jupyter Notebook, but longer term, it should be pulled from an database with the suitable annotations.

## Sub-Population Count vector P[p, 1]

In the original model, we had p sub-populations. In the simplest model, it was just two populations: non-employees of healthcare companies and employees. With SOC and other codes, there are close to a thousands. So the populations are a vector that includes a description and a population count.

So for example P = [ 7400, 435 ] which is just about the right numbers for Seattle and is a [ 1, p ] vector

## Sub-Population Usage of Protection Levels U[p, l]

You can think of this as a one-hot matrix in the most simple form. That is for each subpopulation, what level of protection do you need. 

For example, in the simplest case, if there are six protection levels, then if non-employees get level 1 and healthcare employees get level 6, then the matrix looks like:

     0 1 0 0 0 0 0
     0 0 0 0 0 0 1

While date entry is complicated, this let's you take any given population and give it fractions of protection. For instance, if a healthcare employer typically had 50% of it's workers as office workers at level 2, 25% as customer facing and 10% taking care of non COVID and 15% in direct contact, that vector would look like:

    0 0.5 0.25 0 0 0.10 0.15

This gives the modeler great flexibility with employers or any population

The matrix of usage U looks like p rows and l columns

    Usage_pd.shape = [ p, l ]

## Required Equipment per capita per sub-population R[p, n]

So with this, you can see that with a single operation you can get to the actual equipment levels require per person per day for a given population. Note that there is new Python 3.5 syntax for [matrix multiply](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465)

    U x D = [ p, l ] x [l, n] = R[p, n]
    # in the new Python 3.5 syntax using ampersand
    # np.dot for matrices but not tensors
    R = U @ D
    R = U.dot(D)

In Python Numpy speak, we are doing a matrix [multiply](https://www.tutorialexample.com/understand-numpy-np-multiply-np-dot-and-operation-a-beginner-guide-numpy-tutorial/)

## Total required equipment for a sub-population T[p, n]

Now that we have the per-capita requirements, we need to do a scalar multiply by row

    R[p, n] x P[p, 1] = T[p, n]
    # Or in python using broadcasting which extends P out n columns
    T = R * P

## Merging populations to with Essential index by population E[p, e]

Many times the subpopulations are going to be too large to understand. For instance when there are 800 job classfied by SOC or where there are 350 employer class by NAICS-6, so for convenience, we define essential levels. You can think of the of this as for each population, where do they fit in where they start. Essential (which has changed since version 1.x) can be thought of as the time period of start. So Essential 0 (like Defcon 1), is the most important and so forth. 

This let's you stage start up, so for the example subpopulations of non-healthcare employed and heathcare employed, it might look like a simple matrix across 6 start periods as or more analytically E is p rows and e columns
    0 0 0 0 0 0 1
    1 0 0 0 0 0 0
But this system also allows a stageed restart, so for example, if you want have the workers to come back in the next period for healthcare employees
    0 0 0 0 0 0.5 0.5
    0.5 0.5 0 0 0 0

## Requirements by essential index R[e, n]

In some sense we are doing compression by this, so we are looking at Essential index e is much less than the number of populations p. Or more succinctly e << p and we can get to E with a transpose
    E[p, e].T x T[p, n] = e x p * p x n = R[e, n]
    # In python this looks like
    R = E.T @ T

## The easy parts Required Cost per day RC[e, n] and Stockpile S[e, n]

OK that was the hard stuff, with these matrices reduced to essential levels and the equipment needed for each, there are just some simple scalar multiplies to get where Cost is a row vector that is all the costs

    Gross cost for the equipment = RC[e, n] = R[e, n] * C[1, n]
    Stockpile needed for d Days = S[e, n] = R[e, n] * d

Obviously you may not want to stock pile for all e Essential levels, so you just select what you want for instance S[0] will give you the stockpile needs for the most essential level 0.

## Handling disinfection and reuse








# Detail of Model

These are the details of the model. It is a good example of the parameters that you will need to add. Make sure that you have good advice from medical authorities when looking over these parameters

In [0]:
# Get libraries
import pandas as pd
import numpy as np

In [0]:
#@title Basic Model Parameters
#@markdown Enter the major variables here.

model_name = 'NYC Surge Forecast'  #@param
model_description = 'v1.4 WHO Surge'  #@param {type: "string"}
recurrance_index = 49  #@param {type: "slider", min: 0, max: 100}
recovery_index = 47  #@param {type: "slider", min: 0, max: 100}
revision =   103#@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
model_type = "surge"  #@param ['surge', '3-month', '6-month']
units = "1,000,000" #@param ["1,000,000", "100,000", "10,000", "1,000"] {allow-input: true}
#@markdown ---


## Daily Usage of Equipment __n__ by Protection levels __l__ is D[l, n]

In [0]:
# Eventually we will do this from a database import, but for now, let's use
# the data that is normally in the Excel sheet and just recreate 
# https://colab.research.google.com/drive/1Bcx54NQePYt88RWWmODrRA1pxz-2tnNW?authuser=5#scrollTo=1xwe8g08yRbG

# Using PEP https://www.python.org/dev/peps/pep-0008/
# For simplicity do as a dictionary
Item_name = ['Protection Name',
               'Source of Data',
               'N95 Surgical Respirator',
               'N95 Mask',
               'ASTM 3 Surgical Mask',
               'ASTM 1-2 Surgical Mask',
               'Face Shield',
               'Gown',
               'Gloves',
               'Shoe Covers',
               'Test Kits',
               ''
               ]


Level_name = [ 'WA0', 'WA1', 'WA2', 'WA3', 'WA4', 'WA5', 'WA6']
Item_name = [ 'N95 Surgical', 'non ASTM Mask']
print('Item_names', Item_name)
Daily_usage_matrix = [
                [ 0, 0 ],
                [ 0, 1 ],
                [ 0, 2 ],
                [ 0.1, 3],
                [ 0.2, 4],
                [ 0.3, 6],
                [ 1.18, 0]]

print('Daily_usage_matrix', Daily_usage_matrix)

Daily_usage_df = pd.DataFrame(Daily_usage_matrix,
                              columns = Item_name,
                              index = Level_name)

# use these counts to check the matrix vector bugs
level_count = Daily_usage_df.shape[0]
item_count = Daily_usage_df.shape[1]
print('usage_pd shape is ', Daily_usage_df.shape,
      'protection level count is ', level_count,
      'item count is ', item_count)

print('Daily_usage_pd', Daily_usage_df)

Daily_N95s_usage = Daily_usage_df['N95 Surgical']
print('Daily N95 Surgical Usage', Daily_N95s_usage)

# https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array
print('Daily usage value in Dataframe', Daily_usage_df.values)


Item_names ['N95 Surgical', 'non ASTM Mask']
Daily_usage_matrix [[0, 0], [0, 1], [0, 2], [0.1, 3], [0.2, 4], [0.3, 6], [1.18, 0]]
usage_pd shape is  (7, 2) protection level count is  7 item count is  2
Daily_usage_pd      N95 Surgical  non ASTM Mask
WA0          0.00              0
WA1          0.00              1
WA2          0.00              2
WA3          0.10              3
WA4          0.20              4
WA5          0.30              6
WA6          1.18              0
Daily N95 Surgical Usage WA0    0.00
WA1    0.00
WA2    0.00
WA3    0.10
WA4    0.20
WA5    0.30
WA6    1.18
Name: N95 Surgical, dtype: float64
Daily usage value in Dataframe [[0.   0.  ]
 [0.   1.  ]
 [0.   2.  ]
 [0.1  3.  ]
 [0.2  4.  ]
 [0.3  6.  ]
 [1.18 0.  ]]


## Population Data by sub-populations p is P[p, 1]

Start with the simplest assumption, two populations, one that is `WA6` and one that is `WA2` as an example. But we will insert more data later once we decide the data source.

In [0]:
# This is a dummy test case, later we will use extraction first form a
# spreadsheet and then eventually from a data store that is reliable
# And which has revision control

Population_name = ['Total less healthcare employees', 'Employees of healthcare companies']
Population_data = [7179.6, 735.2 ]

print('Population Data', Population_data)

Population_df = pd.DataFrame(Population_data, index = Population_name, columns = ['Population'])
population_count = Population_df.shape[0]
print('population count p', population_count)
print(Population_df)

# https://note.nkmk.me/en/python-type-isinstance/
print('type of Population_name', type(Population_name))

Population Data [7179.6, 735.2]
population count p 2
                                   Population
Total less healthcare employees        7179.6
Employees of healthcare companies       735.2
type of Population_name <class 'list'>


# Usage of PPE by Sub-population p is U[p, l]

Now we have a vector which are the population usages and we have a list of needs, so we need to do a matrix multiply of population by needs. Each entry is the percentage of a population at a given level.

So in this example, 50% of healthcare workers are level 5 and 50% are at level 6

In [0]:
# Now we need a matrix which is the pop_type x usage_type and the coefficient is just how much is needed for each
# Do this for simplicity start with a zero matrix, we will actually load the data

Usage_by_population_matrix = np.zeros([population_count, level_count])

Usage_by_population_matrix[0,1] = 1.0
Usage_by_population_matrix[1,6] = Usage_by_population_matrix[1, 5] = 0.5
print('Usage_by_population_matrix', Usage_by_population_matrix)

# https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
Usage_by_population_df = pd.DataFrame(Usage_by_population_matrix,
                                      index = Population_name,
                                      columns = Level_name)
print(Usage_by_population_df)



Usage_by_population_matrix [[0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.5]]
                                   WA0  WA1  WA2  WA3  WA4  WA5  WA6
Total less healthcare employees    0.0  1.0  0.0  0.0  0.0  0.0  0.0
Employees of healthcare companies  0.0  0.0  0.0  0.0  0.0  0.5  0.5


# Required Equipment n per capita pre sub-population p is R[p, n]

This is the first multiplication where we take the two matrices and multiply them together.

In [0]:
print('Daily_usage_df', Daily_usage_df.shape)
print('Usage_by_population_df', Usage_by_population_df.shape)

# Note with Panda multiply the index of rows and the columns have to match
Required_df = Usage_by_population_df @ Daily_usage_df

print('Required_df', Required_df)

Daily_usage_df (7, 2)
Usage_by_population_df (2, 7)
Required_df                                    N95 Surgical  non ASTM Mask
Total less healthcare employees            0.00            1.0
Employees of healthcare companies          0.74            3.0


# Attachment: Test Code

Used to test various features of the notebook

## Test of cloning an external Repo

NOte that this does a complete clone in the virtual machine, make sure you have enough space. Also you need to reclone when you close a Notebook instance, so this can be slow with lots of data.

However, it does allow you checkout particular branches and have a realiable dataset.

In [0]:
# Clone the entire repo.
!git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
%cd cloned-repo
!ls

## Test of copying a single file from a repo

This one way to get small datasets, you just point to the raw file and use `!curl` to bring it into the machine.

In [0]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

## Test of connecting to Google Drive

This we can use if we don't need a repo, but are just loading a static file. We normally want everything from a repo or reliable storage, but this is good for quick analysis. In most cases, you should just check this into a repo and then use the github raw extract instead so you get version control.

Note that this does require an authentication everytime you start the Notebook, so the raw extract works better particularly if there it is a public repo.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

## Connecting two cells together for summaries with Cross-output Communications

This is the best method for connecting the longer analysis to a cell that just has the executive summary data. _This does not appear to be working. Need to debug_

In [0]:
%%javascript
const listenerChannel = new BroadcastChannel('channel');
listenerChannel.onmessage = (msg) => {
  const div = document.createElement('div');
  div.textContent = msg.data;
  document.body.appendChild(div);
};

In [0]:
%%javascript
const senderChannel = new BroadcastChannel('channel');
senderChannel.postMessage('Hello world!');

## Creating forms for entry

This is going to be used to parameterize models. This sets global variables that can be used in cells farther down.

In [0]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 142  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


## Display Pandas data dataframes use Vega datasets as an example

This uses the extension `google.colab.data_table` and there is a default data set called `vega_datasets` where you can extract data. It is not clear where the data is or how to figure out how ot use it. Google-fu does not help although the [source code](https://github.com/googlecolab/colabtools/blob/master/google/colab/data_table.py) tells us that `vega_dataset` has airport data in it.

But the hing is in the name Vega which is a visualization package and [Vega Datasets access from Python](https://github.com/jakevdp/vega_datasets) are a standard set of data for visualization testing. The core datasets are kept in [github.io](https://vega.github.io/vega-datasets/).

In [0]:
%load_ext google.colab.data_table

In [0]:
from vega_datasets import data
data.cars()

In [0]:
%unload_ext google.colab.data_table

In [0]:
data.stocks()

## Github Rendering of Jupyter

This is pretty cool, but [Github](https://help.github.com/en/github/managing-files-in-a-repository/working-with-jupyter-notebook-files-on-github) actually renders the Jupyter notebooks as statis HTML when you browse it. That means just clicking on a `.ipynb` will give you something reasonable. It is not interactive nor is anything running behind it, but it does mean that documents produced by use are easily readable.